In [3]:
import pymysql
import pandas as pd
import numpy as np
import datetime
import gc
import sys
sys.path.append(r'D:\jupyter files\waiting_time_project\my_tools')
import tools_for_os.for_df as ml_df

In [2]:
sql_appointment = """SELECT * FROM appointment"""
data_appointment = ml_df.get_df_from_sql(sql_appointment)
print(f'The shape of data_appointment {data_appointment.shape}')
sql_dx = """SELECT * FROM diagnosis"""
data_dx = ml_df.get_df_from_sql(sql_dx)
print(f'The shape of data_dx {data_dx.shape}')
sql_dxt = """SELECT * FROM diagnosistranslation"""
data_dxt = ml_df.get_df_from_sql(sql_dxt)
print(f'The shape of data_dxt {data_dxt.shape}')
sql_pt = """SELECT * FROM patient"""
data_pt = ml_df.get_df_from_sql(sql_pt)
print(f'The shape of data_pt {data_pt.shape}')
sql_ptc = """SELECT * FROM patientcopy"""
data_ptc = ml_df.get_df_from_sql(sql_ptc)
print(f'The shape of data_ptc {data_ptc.shape}')
sql_co = """SELECT * FROM course"""
data_co = ml_df.get_df_from_sql(sql_co)
print(f'The shape of data_co {data_co.shape}')
sql_pl = """SELECT * FROM plan"""
data_pl = ml_df.get_df_from_sql(sql_pl)
print(f'The shape of data_pl {data_pl.shape}')
sql_pd = """SELECT * FROM patientdoctor"""
data_pd = ml_df.get_df_from_sql(sql_pd)
print(f'The shape of data_pd {data_pd.shape}')

The shape of data_appointment (1074478, 17)
The shape of data_dx (39486, 12)
The shape of data_dxt (1291, 6)
The shape of data_pt (51062, 2)
The shape of data_ptc (51308, 6)
The shape of data_co (58360, 11)
The shape of data_pl (148421, 13)
The shape of data_pd (148770, 7)


In [4]:
try:
    data_appointment.drop(columns = ['LastUpdated'], inplace = True)
    data_dx.drop('LastUpdated', axis = 1, inplace = True)
    data_dxt.drop('LastUpdated', axis = 1, inplace = True)
    data_pt.drop('LastUpdated', axis = 1, inplace = True)
    data_ptc.drop('LastUpdated', axis = 1, inplace = True)
    data_co.drop('LastUpdated', axis = 1, inplace = True)
    data_pl.drop('LastUpdated', axis = 1, inplace = True)
    data_pd.drop('LastUpdated', axis = 1, inplace = True)
    print('Drop columns')
except:
    print('Finish droppping columns')

Drop columns


==========================================================================

## Appointment 表

In [4]:
data_appointment.columns

Index(['AppointmentSerNum', 'PatientSerNum', 'AppointmentAriaSer',
       'ActivityInstanceAriaSer', 'DiagnosisSerNum', 'PrioritySerNum',
       'AliasSerNum', 'AliasExpressionSerNum', 'Status', 'State',
       'ScheduledStartTime', 'ScheduledEndTime', 'ActualStartDate',
       'ActualEndDate', 'EstimatedDuration', 'HstryUserName'],
      dtype='object')

In [5]:
data_appointment.ActualStartDate.iloc[0]

datetime.datetime(1970, 1, 1, 0, 0)

In [6]:
print(f'The shape of data_appointment {data_appointment.shape}')
data_appointment = data_appointment[
    (data_appointment.ActualStartDate != datetime.datetime(1970, 1, 1, 0, 0))&
    (data_appointment.ActualEndDate != datetime.datetime(1970, 1, 1, 0, 0))]
print(f'The shape of data_appointment {data_appointment.shape}')

The shape of data_appointment (1074478, 16)
The shape of data_appointment (370303, 16)


In [7]:
data_appointment.Status.value_counts()

Completed           369089
Pt. CompltFinish      1210
                         4
Name: Status, dtype: int64

In [8]:
data_appointment.State.value_counts()

Active     370296
                4
Deleted         3
Name: State, dtype: int64

In [9]:
print(f'The shape of data_appointment {data_appointment.shape}')
data_appointment = data_appointment[data_appointment.State == 'Active']
print(f'The shape of data_appointment {data_appointment.shape}')
data_appointment = data_appointment[(data_appointment.Status == 'Completed') | 
                                    (data_appointment.Status == 'Pt. CompltFinish')]
print(f'The shape of data_appointment {data_appointment.shape}')

The shape of data_appointment (370303, 16)
The shape of data_appointment (370296, 16)
The shape of data_appointment (370296, 16)


## PatientDoctor 表

In [10]:
data_pd.AliasSerNum.value_counts()

37    148770
Name: AliasSerNum, dtype: int64

##### PatientDoctor 表中，AliasSerNum 只有37

##### 筛选出 OncologistFlag = 1，PrimaryFlag = 1 的数据

In [11]:
print(f'The shape of data_pd {data_pd.shape}')
data_pd = data_pd[data_pd.OncologistFlag == 1]
print(f'The shape of data_pd {data_pd.shape}')
data_pd = data_pd[data_pd.PrimaryFlag == 1]
print(f'The shape of data_pd {data_pd.shape}')

The shape of data_pd (148770, 6)
The shape of data_pd (51535, 6)
The shape of data_pd (49874, 6)


## Plan 表

In [12]:
data_pl.head()

,PlanSerNum,CourseSerNum,PlanAriaSer,PlanId,AliasSerNum,AliasExpressionSerNum,TreatmentOrientation,CreationDate,Status,StatusDate,TreatmentType,Intent
0,1,40522,929513,FT1 RT BREAST,37,0,HFS,2015-08-03 09:53:00,TreatApproval,2015-08-05 11:56:00,Linac,CURATIVE
1,2,40438,929546,FT1 RT BREAST,37,0,HFS,2015-08-03 11:40:00,TreatApproval,2015-08-05 14:23:00,Linac,CURATIVE
2,3,40024,922090,FP1 RT BREAST,37,0,HFS,2015-06-25 16:15:00,TreatApproval,2015-06-29 14:52:00,Linac,CURATIVE
3,4,40569,928000,FT1 LT BREAST,37,0,HFS,2015-07-27 15:34:00,Retired,2015-08-04 09:16:00,Linac,CURATIVE
4,5,40644,929664,FT1 LT BREAST,37,0,HFS,2015-08-04 09:56:00,TreatApproval,2015-08-05 13:29:00,Linac,CURATIVE


In [13]:
data_pl.Status.value_counts()

Unapproved          74207
TreatApproval       47300
Rejected            14855
PlanApproval         4944
Retired              3873
CompletedEarly       2550
Completed             458
ExternalApproval      222
Reviewed               12
Name: Status, dtype: int64

In [14]:
data_pl.PlanId.value_counts()

EmergencyPS        7162
1                  1489
FP1 BRAIN          1395
ORIGINAL           1337
FINAL PLAN 1       1310
FP1 PROSTATE       1249
FILMS              1162
FP1 PELVIS         1155
BRAIN              1000
RT BREAST           972
LT BREAST           936
FP1 RBREAST         875
FILM                835
FP1 ENT             829
2                   818
FP1 RT BREAST       789
FP1 LT BREAST       783
CF                  740
FP1 LBREAST         722
PROSTATE            653
PELVIS              626
FP1 RLUNG           619
FP1_BRAIN           588
FP2 LT BREAST       567
FP1 RT LUNG         522
FP1 RECTUM          512
FP2 RT BREAST       474
FP1 LLUNG           468
CFS                 466
PS1                 458
                   ... 
NEW OPT               1
C2M1P2                1
FILM RECTUM REC       1
E Lt Bst              1
FP1LBREASTBHr         1
EXTENDED DIST         1
PL1 Ant/Neck          1
BLADDER MOD           1
C-SPINE-6x            1
FP1LTLG48GNHC         1
QA_FP1_STOMAC   

In [15]:
print(f'The shape of data_pl {data_pl.shape}')
data_pl_ = data_pl[data_pl.Status == 'TreatApproval']
print(f'The shape of data_pl_ {data_pl_.shape}')

The shape of data_pl (148421, 12)
The shape of data_pl_ (47300, 12)


In [16]:
data_pl_.PlanId.value_counts()

FP1 BRAIN        1093
FP1 PROSTATE     1074
FP1 RBREAST       768
FP1 PELVIS        727
FP1 RT BREAST     648
FP1 LBREAST       621
FP1 LT BREAST     615
FP2 LT BREAST     521
FP1 RLUNG         515
FP1_BRAIN         490
FP2 RT BREAST     436
FP2 LBREAST       412
FP1 RT LUNG       390
FP1 RECTUM        376
FP1 LLUNG         370
FP1 ENT           366
FP2 RBREAST       363
FP1 CHEST         327
FP2 PROSTATE      327
FP1 ABDOMEN       305
FP1 LT LUNG       276
FP1 WBI           263
FT1 LT BREAST     241
FP1 PELVIS #      240
FP1_PELVIS        238
FP1 LBREASTBH     230
C1_FP1_BRAIN      206
FT1 RT BREAST     205
FP1 BLADDER       204
FP2 RECTUM        200
                 ... 
DAN NOV 20          1
FP1_DIAPHRAGM       1
RT ILLIAC CRE       1
Vaginal_MOD         1
FP1-RT CHEST        1
Lt Arm Site 2       1
QA_C1_FP1_mc        1
FPL1C1-C4           1
FP1LHEMIPELVI       1
CB_TEST             1
GP1 LARYNX          1
T2                  1
P1 Lat lankle       1
FPL1 T11-S3         1
FP1 L Ches

In [17]:
data_dx.columns

Index(['DiagnosisSerNum', 'PatientSerNum', 'AliasSerNum', 'DiagnosisAriaSer',
       'DiagnosisCreationDate', 'DiagnosisCode', 'Description',
       'HistologyCode', 'HistDescription', 'ObjectStatus', 'DiagnosisType'],
      dtype='object')

In [18]:
data_dx.DiagnosisCode.value_counts()

C50.9     5794
C34.9     3620
C61       3541
C79.5     2164
C79.3     1178
C20       1072
C71.9     1057
C34.1     1050
C61.9      795
C50.4      761
C54.1      704
C85.9      680
C67.9      572
C49.9      564
C50.8      416
C34.3      391
C80        369
C20.9      329
C78.0      321
C64        306
C41.2      305
C18.9      295
C53.9      287
C43.9      285
D05.1      283
D05.9      281
C15.9      281
C44.9      264
C79.4      230
C41.4      226
          ... 
C69.2q       1
D38.4        1
C49.2s       1
348.5        1
724.50       1
C72.2r       1
C38.1        1
C58          1
C51.9s       1
C05.9p       1
C25.1        1
191.3        1
C16.0a       1
719.26       1
C77.0s       1
J98.4        1
C11.3        1
C38.4c       1
789.0        1
C04.8p       1
C53.9p       1
K86.8        1
C50.40       1
191.1        1
C25.3        1
192.1        1
780.7        1
C42.2a       1
M54.5        1
197.7        1
Name: DiagnosisCode, Length: 994, dtype: int64

In [19]:
data_dxt.columns

Index(['DiagnosisTranslationSerNum', 'AliasName', 'WLMAliasName',
       'WLMAliasDetailed', 'DiagnosisCode'],
      dtype='object')

In [20]:
data_dxt.shape

(1291, 5)

==========================================================================

## 数据预处理

In [29]:
data_path = 'D:\\jupyter files\\data_waiting_time_project\\preprocess_data\\'

In [5]:
def data_compare(df1, df2, name):
    df_count1 = df1[name].value_counts().index.tolist()
    df_count2 = df2[name].value_counts().index.tolist()
    same = list(set(df_count1).intersection(set(df_count1)))
    print(f'same: {len(same)}')
    
    a = list(set(df_count1).difference(set(df_count2)))
    print(f'df1 has, df2 does not has: {len(a)}')
    b = list(set(df_count2).difference(set(df_count1)))
    print(f'df2 has, df1 does not has: {len(b)}')

In [6]:
# data_compare(pl_co, pt_pd_appt_dx_dxt, 'DiagnosisSerNum')

In [23]:
print('='*40)
print(f'Start merging data')

# 筛选appointment 中的数据
print('='*20)
print('Process data_appointment')
print(f'\nThe shape of data_appointment {data_appointment.shape}')
data_appointment_ = data_appointment[((data_appointment.AliasSerNum == 31) |
                                     (data_appointment.AliasSerNum == 23)) &
                                     (data_appointment.ActualStartDate != datetime.datetime(1970, 1, 1, 0, 0)) &
                                     (data_appointment.ActualEndDate != datetime.datetime(1970, 1, 1, 0, 0)) &
                                     (data_appointment.ActualStartDate != data_appointment.ActualEndDate)]
print(f'The shape of data_appointment_ {data_appointment_.shape}')

data_appointment_ = data_appointment_[(data_appointment_.State == 'Active') &
                                      ((data_appointment_.Status == 'Completed') |
                                       (data_appointment_.Status == 'Pt. CompltFinish'))]
print(f'The shape of data_appointment_ {data_appointment_.shape}')

# 拼接data_pt 和data_appointment_
print('='*20)
print(f'Merge data_pt and data_appointment_')
print(f'\nThe shape of data_appointment_ {data_appointment_.shape}')
print(f'The shape of data_pt {data_pt.shape}')
pt_appt = pd.merge(data_appointment_, data_pt, on = 'PatientSerNum', how = 'inner')
print(f'pt_appt shape is {pt_appt.shape}')
# del data_appointment_, data_pt
# gc.collect()

print(f'\nMerge data_ptc')
print(f'\nThe shape of data_ptc {data_ptc.shape}')
pt_appt = pd.merge(pt_appt, data_ptc, on = 'PatientSerNum', how = 'inner')
print(f'pt_appt shape is {pt_appt.shape}')
# del data_ptc
# gc.collect()

# PatientDoctor 的预处理
print('='*20)
print('Process data_pd')
print(f'\nThe shape of data_pd {data_pd.shape}')
data_pd_ = data_pd[(data_pd.OncologistFlag == 1) &
                   (data_pd.PrimaryFlag == 1)]
print(f'The shape of data_pd_ {data_pd_.shape}')

# 拼接pt_appt 和data_pd_
print('='*20)
print(f'Merge data_pd_ and pt_appt')
# data_pd_, data_appt AliasSerNum 不同，data_appt_ 中只保留了23、31，data_pd_ 中只有37
data_pd_.drop('AliasSerNum', axis = 1, inplace = True)
pt_pd_appt = pd.merge(pt_appt, data_pd_, on = 'PatientSerNum', how = 'left')
print(f'\npt_pd_appt shape is {pt_pd_appt.shape}')
# del pt_appt, data_pd_
# gc.collect()

# 拼接data_dxt 和data_dx
print('='*20)
print(f'Merge data_dxt and data_dx')
dx_dxt_ = data_dxt.rename(columns={'AliasName': 'dxt_AliasName'}, inplace = False) 
print(f'\nThe shape of data_dx {data_dx.shape}')
print(f'The shape of dx_dxt_ {dx_dxt_.shape}')
dx_dxt = pd.merge(data_dx, dx_dxt_, on = 'DiagnosisCode', how = 'left')
print(f'dx_dxt shape is {dx_dxt.shape}')

# 拼接pt_appt 和dx_dxt
print(f'\nMerge pt_appt and dx_dxt')
dx_dxt.drop('AliasSerNum', axis = 1, inplace = True)
dx_dxt.drop('PatientSerNum', axis = 1, inplace = True)
pt_pd_appt_dx_dxt = pd.merge(pt_pd_appt, dx_dxt, on = ['DiagnosisSerNum'], how = 'inner')
print(f'\npt_pd_appt_dx_dxt shape is {pt_pd_appt_dx_dxt.shape}')

# Plan 的预处理
print('='*20)
print('Process data_pl')
print(f'\nThe shape of data_pl {data_pl.shape}')
data_pl_ = data_pl[data_pl.Status == 'TreatApproval']
data_pl_.TreatmentOrientation = data_pl_.TreatmentOrientation.apply(
    lambda x: 'NULL' if x == '' else x)
print(f'The shape of data_pl_ {data_pl_.shape}')

# 拼接data_pl_ 和data_co
print('='*20)
print(f'Merge data_pl_ and data_co')
data_pl_.drop('AliasSerNum', axis = 1, inplace = True)
data_co_ = data_co.drop('AliasSerNum', axis = 1, inplace = False)
print(f'\nThe shape of data_pl_ {data_pl_.shape}')
print(f'The shape of data_co_ {data_co_.shape}')
pl_co = pd.merge(data_pl_, data_co_, on = 'CourseSerNum', how = 'inner')
print(f'pl_co shape is {pl_co.shape}')

print(f'\nMerge pt_pd_appt_dx_dxt and pl_co')
pl_co.drop('AliasExpressionSerNum', axis = 1, inplace = True)
pl_co.drop('DiagnosisSerNum', axis = 1, inplace = True)
pl_co.drop('PrioritySerNum', axis = 1, inplace = True)
pl_co.rename(columns={'Status': 'Status_plan'}, inplace=True)
pt_pd_appt_dx_dxt_pl_co = pd.merge(pt_pd_appt_dx_dxt, pl_co,
                                   on = ['PatientSerNum'], 
                                   how = 'inner')
print(f'\npt_pd_appt_dx_dxt_pl_co shape is {pt_pd_appt_dx_dxt_pl_co.shape}')

# print(pt_pd_appt_dx_dxt_pl_co.columns)

#　删除值完全相同的列
print('='*20)
print(f'Drop columns with same values')
for col in pt_pd_appt_dx_dxt_pl_co.columns:
    if len(pt_pd_appt_dx_dxt_pl_co[col].unique()) == 1:
        pt_pd_appt_dx_dxt_pl_co.drop(col, axis = 1, inplace = True)
print(f'\npt_pd_appt_dx_dxt_pl_co shape {pt_pd_appt_dx_dxt_pl_co.shape}')

# print(pt_pd_appt_dx_dxt_pl_co.columns)


Start merging data
Process data_appointment

The shape of data_appointment (1074478, 16)
The shape of data_appointment_ (367413, 16)
The shape of data_appointment_ (367410, 16)
Merge data_pt and data_appointment_

The shape of data_appointment_ (367410, 16)
The shape of data_pt (51062, 1)
pt_appt shape is (367410, 16)

Merge data_ptc

The shape of data_ptc (51308, 5)
pt_appt shape is (367410, 20)
Process data_pd

The shape of data_pd (148770, 6)
The shape of data_pd_ (49874, 6)
Merge data_pd_ and pt_appt

pt_pd_appt shape is (410947, 24)
Merge data_dxt and data_dx

The shape of data_dx (39486, 11)
The shape of dx_dxt_ (1291, 5)
dx_dxt shape is (51712, 15)

Merge pt_appt and dx_dxt

pt_pd_appt_dx_dxt shape is (459642, 36)
Process data_pl

The shape of data_pl (148421, 12)
The shape of data_pl_ (47300, 12)
Merge data_pl_ and data_co

The shape of data_pl_ (47300, 11)
The shape of data_co_ (58360, 9)
pl_co shape is (47300, 19)

Merge pt_pd_appt_dx_dxt and pl_co

pt_pd_appt_dx_dxt_pl_co sh

In [25]:
feature_columns = [
    'dxt_AliasName',
    'DateOfBirth', 'Sex',
    'AliasSerNum', 'PatientSerNum', 'AppointmentSerNum',
    'ScheduledStartTime', 'ScheduledEndTime', 'ActualStartDate', 'ActualEndDate',
    'CourseSerNum', 
    'DoctorSerNum',
    'PlanSerNum',
    'TreatmentOrientation'
]

data_part1 = pt_pd_appt_dx_dxt_pl_co[feature_columns]

In [27]:
# 患者年龄
data_part1['age'] = data_part1.apply(lambda x: int((x.ActualStartDate - x.DateOfBirth).days/365), axis = 1)

# 时间相关的特征，月-日-周-小时
data_part1['month'] = data_part1.apply(lambda x: x.ScheduledStartTime.strftime("%m"), axis = 1)
data_part1['date'] = data_part1.apply(lambda x: x.ScheduledStartTime.strftime("%Y--%m--%d"), axis = 1)
data_part1['week'] = data_part1.apply(lambda x: x.ScheduledStartTime.strftime("%w"), axis = 1)
data_part1['hour'] = data_part1.apply(lambda x: x.ScheduledStartTime.strftime("%H"), axis = 1)

# 时长相关特征
data_part1['Scheduled_duration'] = data_part1.apply(lambda x: (x.ScheduledEndTime - x.ScheduledStartTime).seconds/60, axis = 1)
data_part1['Actual_duration'] = data_part1.apply(lambda x: (x.ActualEndDate - x.ActualStartDate).seconds/60, axis = 1)

data_part1.sort_values(by = ['PatientSerNum', 'AppointmentSerNum'], inplace = True)

C:\Users\WENDY\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\WENDY\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\WENDY\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [28]:
data_part1.head()

,dxt_AliasName,DateOfBirth,Sex,AliasSerNum,PatientSerNum,AppointmentSerNum,ScheduledStartTime,ScheduledEndTime,ActualStartDate,ActualEndDate,...,DoctorSerNum,PlanSerNum,TreatmentOrientation,age,month,date,week,hour,Scheduled_duration,Actual_duration
0,Breast,1948-09-13 00:00:00,Female,31,18,567562,2008-05-09 10:15:00,2008-05-09 10:30:00,2008-05-09 09:20:00,2008-05-09 09:53:00,...,6376.0,43453,HFS,59,05,2008--05--09,5,10,15.0,33.0
1,Breast,1931-04-02 00:00:00,Female,23,67,567569,2010-04-09 09:15:00,2010-04-09 09:30:00,2010-04-09 09:15:00,2010-04-09 09:28:00,...,5691.0,57293,HFS,79,04,2010--04--09,5,09,15.0,13.0
2,Breast,1931-04-02 00:00:00,Female,23,67,567570,2010-04-12 09:15:00,2010-04-12 09:30:00,2010-04-12 08:57:00,2010-04-12 09:09:00,...,5691.0,57293,HFS,79,04,2010--04--12,1,09,15.0,12.0
3,Breast,1931-04-02 00:00:00,Female,23,67,567571,2010-04-13 09:15:00,2010-04-13 09:30:00,2010-04-13 09:40:00,2010-04-13 09:56:00,...,5691.0,57293,HFS,79,04,2010--04--13,2,09,15.0,16.0
4,Breast,1931-04-02 00:00:00,Female,23,67,567572,2010-04-14 09:15:00,2010-04-14 09:30:00,2010-04-14 09:06:00,2010-04-14 09:46:00,...,5691.0,57293,HFS,79,04,2010--04--14,3,09,15.0,40.0


In [179]:
data_part1_grouped = data_part1.groupby('PatientSerNum')
sample_data = data_part1_grouped.get_group(67)

In [30]:
data_part1.to_csv(data_path + 'data_part1.csv')